In [1]:
#load packages
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [2]:
rw = pd.read_csv("../Datasets/NEW/RW_with_airports.csv", parse_dates=['date'])

In [3]:
rw.head()

,tail_number,date,aircraft,origin_code,origin,destination_code,destination,departure,arrival,duration,distance_mi,Owner,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,airport_origin,city_origin,origin_state,country_origin,airport_destination,city_destination,destination_state,country_destination,departure_shift,arrival_shift
0,N640EC,2021-03-11,EC35,KPGV,Pitt-Greenville (KPGV),KRWI,Rocky Mount-Wilson Rgnl (KRWI),04:29AM EST,04:46AM EST,16,32.296705,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",35.635201,-77.385300,35.856300,-77.891899,Pitt Greenville Airport,Greenville,NC,US,Rocky Mount Wilson Regional Airport,Rocky Mount,NC,US,night,night
1,N640EC,2021-03-11,EC35,KRWI,Rocky Mount-Wilson Rgnl (KRWI),KPGV,Pitt-Greenville (KPGV),02:17AM EST,03:47AM EST,89,32.296705,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",35.856300,-77.891899,35.635201,-77.385300,Rocky Mount Wilson Regional Airport,Rocky Mount,NC,US,Pitt Greenville Airport,Greenville,NC,US,night,night
2,N640EC,2021-03-11,EC35,KPGV,Pitt-Greenville (KPGV),KRWI,Rocky Mount-Wilson Rgnl (KRWI),01:15AM EST,01:30AM EST,15,32.296705,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",35.635201,-77.385300,35.856300,-77.891899,Pitt Greenville Airport,Greenville,NC,US,Rocky Mount Wilson Regional Airport,Rocky Mount,NC,US,night,night
3,N640EC,2021-03-10,EC35,KIXA,"Roanoke Rapids, NC",KPGV,Pitt-Greenville (KPGV),11:33PM EST,12:02AM EST,28,54.850847,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",36.403840,-77.637860,35.635201,-77.385300,Halifax-Northampton Regional Airport,Roanoke Rapids,NC,US,Pitt Greenville Airport,Greenville,NC,US,night,night
4,N640EC,2021-03-10,EC35,KRWI,Rocky Mount-Wilson Rgnl (KRWI),KIXA,"Roanoke Rapids, NC",08:50PM EST,09:31PM EST,41,41.163189,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",35.856300,-77.891899,36.413910,-77.628940,Rocky Mount Wilson Regional Airport,Rocky Mount,NC,US,Halifax-Northampton Regional Airport,Roanoke Rapids,NC,US,night,night


In [4]:
rw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96074 entries, 0 to 96073
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   tail_number            96074 non-null  object        
 1   date                   96074 non-null  datetime64[ns]
 2   aircraft               90671 non-null  object        
 3   origin_code            96074 non-null  object        
 4   origin                 96074 non-null  object        
 5   destination_code       96074 non-null  object        
 6   destination            96074 non-null  object        
 7   departure              96074 non-null  object        
 8   arrival                96074 non-null  object        
 9   duration               96074 non-null  int64         
 10  distance_mi            96074 non-null  float64       
 11  Owner                  96074 non-null  object        
 12  origin_Latitude        96074 non-null  float64       
 13  o

In [5]:
rw.isna().sum()

tail_number                 0
date                        0
aircraft                 5403
origin_code                 0
origin                      0
destination_code            0
destination                 0
departure                   0
arrival                     0
duration                    0
distance_mi                 0
Owner                       0
origin_Latitude             0
origin_Longitude            0
destination_Latitude        0
destination_Longitude       0
airport_origin              0
city_origin                63
origin_state               40
country_origin              0
airport_destination         0
city_destination            0
destination_state          23
country_destination         0
departure_shift             0
arrival_shift               0
dtype: int64

In [6]:
# set Owner and tail_number as indexes
rw = rw.set_index(['Owner', 'tail_number'])

In [7]:
# calculate 25 quantile of each tail number of the airmedical service
rw['duration_q_25'] = rw.groupby(['Owner', 'tail_number']).duration.quantile(0.25)

# calculate 75 quantile of each tail number of the airmedical service
rw['duration_q_75'] = rw.groupby(['Owner', 'tail_number']).duration.quantile(0.75)

# calculate IQR
rw['duration_iqr'] = rw['duration_q_75'] - rw['duration_q_25']

# calculate lower whisker
rw['duration_lower_whisker'] = rw['duration_q_25'] - (1.5 * rw['duration_iqr'])

# calculate upper whisker
rw['duration_upper_whisker'] = rw['duration_q_75'] + (1.5 * rw['duration_iqr'])

In [8]:
rw['duration_outlier'] = np.where((rw.duration < rw.duration_lower_whisker) | (rw.duration > rw.duration_upper_whisker), "Yes", "No")

In [9]:
# calculate 25 quantile of each tail number of the airmedical service
rw['distance_q_25'] = rw.groupby(['Owner', 'tail_number']).distance_mi.quantile(0.25)

# calculate 75 quantile of each tail number of the airmedical service
rw['distance_q_75'] = rw.groupby(['Owner', 'tail_number']).distance_mi.quantile(0.75)

# calculate IQR
rw['distance_iqr'] = rw['distance_q_75'] - rw['distance_q_25']

# calculate lower whisker
rw['distance_lower_whisker'] = rw['distance_q_25'] - (1.5 * rw['distance_iqr'])

# calculate upper whisker
rw['distance_upper_whisker'] = rw['distance_q_75'] + (1.5 * rw['distance_iqr'])

In [10]:
rw['distance_outlier'] = np.where((rw.distance_mi < rw.distance_lower_whisker) | (rw.distance_mi > rw.distance_upper_whisker), "Yes", "No")

In [11]:
rw.head()

date  \
Owner                                             tail_number              
METRO AVIATION INCSHREVEPORT, LA, US(Corporation) N640EC      2021-03-11   
                                                  N640EC      2021-03-11   
                                                  N640EC      2021-03-11   
                                                  N640EC      2021-03-10   
                                                  N640EC      2021-03-10   

                                                              aircraft  \
Owner                                             tail_number            
METRO AVIATION INCSHREVEPORT, LA, US(Corporation) N640EC          EC35   
                                                  N640EC          EC35   
                                                  N640EC          EC35   
                                                  N640EC          EC35   
                                                  N640EC          EC35   

                                                              origin_code  \
Owner                                             tail_number               
METRO AVIATION INCSHREVEPORT, LA, US(Corporation) N640EC             KPGV   
                                                  N640EC             KRWI   
                                                  N640EC             KPGV   
                                                  N640EC             KIXA   
                                                  N640EC             KRWI   

                                                                                       origin  \
Owner                                             tail_number                                   
METRO AVIATION INCSHREVEPORT, LA, US(Corporation) N640EC               Pitt-Greenville (KPGV)   
                                                  N640EC       Rocky Mount-Wilson Rgnl (KRWI)   
                                                  N640EC               Pitt-Greenville (KPGV)   
                                                  N640EC                   Roanoke Rapids, NC   
                                                  N640EC       Rocky Mount-Wilson Rgnl (KRWI)   

                                                              destination_code  \
Owner                                             tail_number                    
METRO AVIATION INCSHREVEPORT, LA, US(Corporation) N640EC                  KRWI   
                                                  N640EC                  KPGV   
                                                  N640EC                  KRWI   
                                                  N640EC                  KPGV   
                                                  N640EC                  KIXA   

                                                                                  destination  \
Owner                                             tail_number                                   
METRO AVIATION INCSHREVEPORT, LA, US(Corporation) N640EC       Rocky Mount-Wilson Rgnl (KRWI)   
                                                  N640EC               Pitt-Greenville (KPGV)   
                                                  N640EC       Rocky Mount-Wilson Rgnl (KRWI)   
                                                  N640EC               Pitt-Greenville (KPGV)   
                                                  N640EC                   Roanoke Rapids, NC   

                                                                 departure  \
Owner                                             tail_number                
METRO AVIATION INCSHREVEPORT, LA, US(Corporation) N640EC       04:29AM EST   
                                                  N640EC       02:17AM EST   
                                                  N640EC       01:15AM EST   
                                                  N640EC       11:33PM EST   
                                                  N640EC       08:50PM ES

In [12]:
rw.to_csv('..//Datasets/NEW/RW_Outliers.csv')